In [1]:
from pandas import DataFrame
from operator import itemgetter
from collections import defaultdict
from itertools import chain
import pandas as pd
import numpy as np
import requests
import pprint
import json
from pandas.io.json import json_normalize
import xmltodict
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

### 교통사고 구급출동 현황 
- 중증질환 데이터와 같은 방식으로 다시 전처리
1. 병원명1 null, 시간 관련 데이터 null, 위치 정보 null
2. 위도, 경도 생성
3. 전북 응급실 병원 목록과 사건 현장 위도 경도에 따라 병원별 사건당 거리 
4. 소방센터 새로운 버전으로 만든 센터위도경도 데이터 사용해서 센터별 사건당 거리 

텍스트 연관..? 동일 텍스트가 포함된 데이터만 출력해줌 

### 예상 분석 방향 시나리오
- 그 병원들 중에서 응급실, 입원실등의 잔여 병상 수 파악하여 혼잡도 생성 (다른 구급차와 소통하며 실시간 병원 선택에 따른 혼잡도 실시간 갱신 된 데이터를 기반으로 혼잡도 값 생성)

- 병원 가까운 순으로 정렬 

- 위의 두 값을 결합하여 환자 응급 증상에 따른 최적 병원 출력


1. 구급출동 데이터 
2. 응급실 병원 목록 우선 결합할 때는 전북지역의 병원으로 거리 생성
3. 소방센터 위도 경도 데이터 

In [5]:
car_acdt = pd.read_csv('/Users/subikim/Desktop/연구실/연구실자료/응급플랫폼논문/교통사고 구급출동 현황_2011_2020.csv',)

In [8]:
car_acdt = car_acdt.drop(0)

In [11]:
len(car_acdt)

110381

In [15]:
car_acdt = car_acdt.dropna(subset=['신고일자',
 '신고시각',
 '신고연도','신고월',
 '신고일',
 '신고시',
 '신고분',
 '요일',
 '출동일자',
 '출동시각',
 '출동연도',
 '출동월',
 '출동일',
 '출동시',
 '출동분',
 '현장도착일자',
 '현장도착시각',
 '현장도착연도',
 '현장도착월',
 '현장도착일',
 '현장도착시',
 '현장도착분',
 '환자접촉일자',
 '환자접촉시각',
 '환자접촉연도',
 '환자접촉월',
 '환자접촉일',
 '환자접촉시',
 '환자접촉분',
 '현장거리',
 '현장출발일자',
 '현장출발시각',
 '현장출발연도',
 '현장출발월',
 '현장출발일',
 '현장출발시',
 '현장출발분',
 '병원도착일자',
 '병원도착시각',
 '병원도착연도',
 '병원도착월',
 '병원도착일',
 '병원도착시', '병원도착분','귀소일자', '귀소시각', '귀소연도', '귀소월', '귀소일', '귀소시','귀소분','위치정보X','병원명1'])

In [19]:
car_acdt = car_acdt[car_acdt['신고연도'].isin([2018,2019,2020])].reset_index(drop=True)

In [22]:
ex = car_acdt.copy()

In [23]:
import pyproj
import folium

df = ex[['위치정보X','위치정보Y']]

df

coord = np.array(df)
coord

def project_array(coord, p1_type, p2_type):
    """
    좌표계 변환 함수
    - coord: x, y 좌표 정보가 담긴 NumPy Array
    - p1_type: 입력 좌표계 정보 ex) epsg:5179
    - p2_type: 출력 좌표계 정보 ex) epsg:4326
    """
    p1 = pyproj.Proj(init=p1_type)
    p2 = pyproj.Proj(init=p2_type)
    fx, fy = pyproj.transform(p1, p2, coord[:, 0], coord[:, 1])
    return np.dstack([fx, fy])[0]

p1_type = "epsg:2097"
p2_type = "epsg:4326"

# project_array() 함수 실행
result = project_array(coord, p1_type, p2_type)
result

ex['경도'] = result[:, 0]
ex['위도'] = result[:, 1]

In [25]:
car_acdt = ex.copy()

In [34]:
# 사건 당 병원별 거리 구함 
import geopy.distance

In [70]:
hspt = pd.read_csv('/Users/subikim/Desktop/연구실/연구실자료/응급플랫폼논문/전북실시간응급실병원.csv',
                        index_col=[0])

In [37]:
hspt = hspt[['dutyName','dutyAddr','wgs84Lat','wgs84Lon','위도_경도']]
hspt

,dutyName,dutyAddr,wgs84Lat,wgs84Lon,위도_경도
0,의료법인석천재단고창병원,전라북도 고창군 고창읍 화신1길 9,35.436451,126.693318,"35.43645131091079,126.69331814963527"
1,전라북도군산의료원,"전라북도 군산시 의료원로 27 (지곡동, 군산의료원)",35.954971,126.711899,"35.95497072788432,126.71189936455204"
2,의료법인오성의료재단동군산병원,전라북도 군산시 조촌로 149 (조촌동),35.977401,126.737192,"35.97740074466571,126.73719225835914"
3,김제우석병원,전라북도 김제시 서암4길 45 (서암동),35.806739,126.879700,"35.80673935689978,126.8796995613974"
4,전라북도 남원의료원,"전라북도 남원시 충정로 365 (고죽동,(고죽동))",35.431899,127.407191,"35.43189945917936,127.40719103150715"
5,무주군보건의료원,"전라북도 무주군 무주읍 한풍루로 413, 무주군보건의료원",36.005863,127.671703,"36.00586316008286,127.6717026515394"
6,의료법인혜성병원,"전라북도 부안군 부안읍 부령로 33,",35.732775,126.746139,"35.73277494145881,126.74613942623456"
7,부안성모병원,전라북도 부안군 부안읍 오정2길 24 (부안읍),35.724815,126.734526,"35.72481539925256,126.73452574138332"
8,순창군보건의료원,전라북도 순창군 순창읍 교성로 135 (순창군보건의료원),35.367150,127.143867,"35.36714953529629,127.14386670292174"
9,원광대학교병원,전라북도 익산시 무왕로 895 (신동),35.964242,126.959306,"35.9642419612473,126.959306346821"


In [38]:
hspt.columns=['병원명','병원주소','위도','경도','위도_경도']

In [39]:
hspt_위도 = hspt['위도'].tolist()
hspt_경도 = hspt['경도'].tolist()
hspt_경도

[126.69331814963527,
 126.71189936455204,
 126.73719225835914,
 126.8796995613974,
 127.40719103150715,
 127.6717026515394,
 126.74613942623456,
 126.73452574138332,
 127.14386670292174,
 126.959306346821,
 126.96649966881398,
 127.28697578028248,
 127.52014287444382,
 127.1393674038135,
 127.153316348062,
 127.14790290638275,
 127.13325187439274,
 127.1247991889436,
 126.82346011194952,
 127.43207008323516]

In [41]:
df = car_acdt.copy()

In [45]:
def hspt_distance(name,num):
    df[name] = [geopy.distance.geodesic((df['위도'].iloc[i],
    df['경도'].iloc[i]),(hspt_위도[num],hspt_경도[num])).km for i in range(len(df))]
    return df

In [47]:
hspt_name = hspt['병원명'].tolist()

In [48]:
for j in range(len(hspt)):
    hspt_distance(hspt_name[j],j)
    print(j)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [50]:
df.head()

,구급보고서번호,등록완료구분명,환자발생유형구분명,교통사고명,교통사고기타명,소방서명,서센터명,소방지역대명,구급활동구분명,헬리콥터펌뷸런스유무명,신고일자,신고시각,신고연도,계절구분명,분기구분,신고월,신고일,신고시,신고분,요일,출동일자,출동시각,출동연도,출동월,출동일,출동시,출동분,현장도착일자,현장도착시각,현장도착연도,현장도착월,현장도착일,현장도착시,현장도착분,환자접촉일자,환자접촉시각,환자접촉연도,환자접촉월,환자접촉일,환자접촉시,환자접촉분,현장거리,현장출발일자,현장출발시각,현장출발연도,현장출발월,현장출발일,현장출발시,현장출발분,병원도착일자,병원도착시각,병원도착연도,병원도착월,병원도착일,병원도착시,병원도착분,귀소일자,귀소시각,귀소연도,귀소월,귀소일,귀소시,귀소분,긴급구조종결구분명,환자연령,환자성별구분명,환자시도명,환자시군구명,환자읍면동명,시도명,시군구명,읍면동명,도시농촌구분명,읍면동구분명,위치정보X,위치정보Y,환자직업명,사고발생장소,병원명1,병원도착일자1,병원도착시각1,병원인계일자1,병원인계시각1,현장거리1,의료기관선정기타병원명1,의료기관선정기타병원명2,재이송사유1,병원명2,병원도착일자2,병원도착시각2,병원인계일자2,병원인계시각2,현장거리2,재이송사유2,이송분류명,이송분류사유,시간단위기온,시간단위강수량,시간단위풍속,시간단위풍향,시간단위습도,시간단위적설량,경도,위도,의료법인석천재단고창병원,전라북도군산의료원,의료법인오성의료재단동군산병원,김제우석병원,전라북도 남원의료원,무주군보건의료원,의료법인혜성병원,부안성모병원,순창군보건의료원,원광대학교병원,의료법인대산의료재단익산병원,임실군보건의료원,장수군보건의료원,전북대학교병원,대자인병원,의료법인영경의료재단호성전주병원,재단법인예수병원유지재단예수병원,의료법인영경의료재단전주병원,재단법인아산사회복지재단정읍아산병원,진안군의료원
0,20184502108M00002,등록완료,질병외,동승자,NaN,전주완산소방서,서부119안전센터,NaN,특수일반,펌뷸런스,20180101,5300,2018,겨울,1,1,1,0,53,월요일,20180101,5740,2018,1,1,0,57,20180101,5853,2018,1,1,0,58,20180101.0,5900.0,2018.0,1.0,1.0,0.0,59.0,1.0,20180101.0,10100.0,2018.0,1.0,1.0,1.0,1.0,20180101.0,10800.0,2018.0,1.0,1.0,1.0,8.0,20180101.0,12400.0,2018.0,1.0,1.0,1.0,24.0,정상,40.0,여,전라북도,전주시덕진구,송천동1가,전라북도,전주시완산구,효자동2가,도시,동,209106.3906,256906.9109,기타,도로,재단법인예수병원유지재단예수병원,20180101.0,10800.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,이송,NaN,0.4,NaN,0.8,110.0,61.0,NaN,127.098673,35.812219,55.557549,38.348077,37.429986,19.800166,50.613768,56.010689,33.073028,34.326878,49.550608,21.044613,20.431633,27.953708,41.945082,5.328294,6.198508,7.400699,3.134458,2.381629,35.125704,30.193808
1,20184504104M00001,등록완료,질병외,보행자,NaN,익산소방서,모현119안전센터,NaN,특수일반,펌뷸런스,20180101,31900,2018,겨울,1,1,1,3,19,월요일,20180101,32236,2018,1,1,3,22,20180101,32500,2018,1,1,3,25,20180101.0,32700.0,2018.0,1.0,1.0,3.0,27.0,2.0,20180101.0,32800.0,2018.0,1.0,1.0,3.0,28.0,20180101.0,33400.0,2018.0,1.0,1.0,3.0,34.0,20180101.0,34700.0,2018.0,1.0,1.0,3.0,47.0,정상,18.0,남,NaN,NaN,NaN,전라북도,익산시,신동,도시,동,196130.6384,273453.8304,학생,도로,(의료법인 대산의료재단) 익산병원,20180101.0,33400.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,이송,NaN,-0.6,NaN,0.6,360.0,63.0,NaN,126.955019,35.961381,62.874428,21.943629,19.727865,18.457182,71.596965,64.820265,31.614158,32.951045,68.112761,0.500392,1.036205,48.981077,61.375786,20.930625,22.008285,20.401630,22.911648,22.331624,42.996223,46.859504
2,20184504104M00002,등록완료,질병외,보행자,NaN,익산소방서,모현119안전센터,NaN,특수일반,펌뷸런스,20180101,31900,2018,겨울,1,1,1,3,19,월요일,20180101,32236,2018,1,1,3,22,20180101,32500,2018,1,1,3,25,20180101.0,32600.0,2018.0,1.0,1.0,3.0,26.0,2.0,20180101.0,32800.0,2018.0,1.0,1.0,3.0,28.0,20180101.0,33400.0,2018.0,1.0,1.0,3.0,34.0,20180101.0,34700.0,2018.0,1.0,1.0,3.0,47.0,정상,18.0,남,NaN,NaN,NaN,전라북도,익산시,신동,도시,동,196130.6384,273453.8304,학생,도로,(의료법인 대산의료재단) 익산병원,20180101.0,33400.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,이송,NaN,-0.6,NaN,0.6,360.0,63.0,NaN,126.955019,35.961381,62.874428,21.943629,19.727865,18.457182,71.596965,64.820265,31.614158,32.951045,68.112761,0.500392,1.036205,48.981077,61.375786,20.930625,22.008285,20.401630,22.911648,22.331624,42.996223,46.859504
3,20184503105M00001,등록완료,질병외,운전자,NaN,군산소방서,경암119안전센터,NaN,특수일반,펌뷸런스,20180101,32500,2018,겨울,1,1,1,3,25,월요일,20180101,32816,2018,1,1,3,28,20180101,32900,2018,1,1,3,29,20180101.0,32900.0,2018.0,1.0,1.0,3.0,29.0,1.5,20180101.0,33700.0,2018.0,1.0,1.0,3.0,37.0,20180101.0,33800.0,2018.0,1.0,1.0,3.0,38.0,20180101.0,35000.0,2018.0,1.0,1.0,3.0,50.0,정상,27.0,남,NaN,NaN,NaN,전라북도,군산시,구암동,도시,동,176481.54,276113.62,학생,도로,의료법인 오성의료재단 동군산병원,20180101.0,33800.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,이송,NaN,-0.3,NaN,1.4,50.0,66.0,NaN,126.737122,35.985075,61.000668,4.041406,0.851496,23.605954,86.278197,84.301340,28.006098,28.878345,77.823263,20.172283,20.850720,64.647306,79.692990,39.407575,40.611888,39.374006,40.466321,39.760455,44.634813,66.188687
4,20184502102M00004,등록완료,질병외,보행자,NaN,전주완산소방서,효자119안전센터,NaN,특수전문,NaN,20180101,41000,2018,겨울,1,1,1,4,10,월요일,20180101,413

### 안전센터 거리 

In [51]:
center = pd.read_csv('/Users/subikim/Desktop/연구실/연구실자료/응급플랫폼논문/안전센터위도경도결합_new2.csv')

In [52]:
center.head()

,안전센터명,위도_경도,위도,경도
0,고창119안전센터,"35.43753836705514,126.6966403334027",35.437538,126.696640
1,대산119안전센터,"35.342566108357076,126.60741090526504",35.342566,126.607411
2,경암119안전센터,"35.981868616333415,126.73552506918483",35.981869,126.735525
3,금동119안전센터,"35.98987412249877,126.70716842500765",35.989874,126.707168
4,대야119안전센터,"35.94622690898768,126.82008419801927",35.946227,126.820084


In [53]:
def center_distance(name,num):
    center_위도 = center['위도'].tolist()
    center_경도 = center['경도'].tolist()
    df[name] = [geopy.distance.geodesic((df['위도'].iloc[i],
    df['경도'].iloc[i]),(center_위도[num],center_경도[num])).km for i in range(len(df))]
    return df

In [54]:
center_name = center['안전센터명'].tolist()
center_name

['고창119안전센터',
 '대산119안전센터',
 '경암119안전센터',
 '금동119안전센터',
 '대야119안전센터',
 '지곡119안전센터',
 '비응119안전센터',
 '항만119안전센터',
 '금산119안전센터',
 '무주119안전센터',
 '격포119안전센터',
 '부안119안전센터',
 '순창119안전센터',
 '고산119안전센터',
 '삼례119안전센터',
 '봉동119안전센터',
 '금마119안전센터',
 '팔봉119안전센터',
 '모현119안전센터',
 '공단119안전센터',
 '남중119안전센터',
 '함열119안전센터',
 '임실119안전센터',
 '장수119안전센터',
 '전미119안전센터',
 '팔복119안전센터',
 '아중119안전센터',
 '교동119안전센터',
 '노송119안전센터',
 '서부119안전센터',
 '평화119안전센터',
 '진안119안전센터',
 '무장119안전센터',
 '흥덕119안전센터',
 '소룡119안전센터',
 '사정119안전센터',
 '만경119안전센터',
 '금지119안전센터',
 '식정119안전센터',
 '인월119안전센터',
 '혁신119안전센터',
 '인화119안전센터',
 '장계119안전센터',
 '금암119안전센터',
 '효자119안전센터',
 '신태인119안전센터',
 '영원119안전센터',
 '마령119안전센터',
 '하북119안전센터',
 '시기119안전센터',
 '구조구급과',
 '소방행정과']

In [59]:
for j in range(len(center)):
    center_distance(center_name[j],j)
    print(j)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51


In [61]:
df.to_csv('/Users/subikim/Desktop/연구실/연구실자료/응급플랫폼논문/교통사고구급출동_응급실병원_안전센터결합(0317).csv',
         index=False)

In [85]:
df.head(10).to_excel('/Users/subikim/Desktop/연구실/연구실자료/응급플랫폼논문/교통사고_병원_센터결합(10).xlsx',
             index=False)

In [62]:
severe_hspt_all = pd.read_csv('/Users/subikim/Desktop/연구실/연구실자료/응급플랫폼논문/중증질환수용가능병원(전국).csv')

In [64]:
severe_hspt_jb = pd.read_csv('/Users/subikim/Desktop/연구실/연구실자료/응급플랫폼논문/중증질환수용가능병원(전북).csv')

In [87]:
severe_hspt_jb.head(10).to_excel('/Users/subikim/Desktop/연구실/연구실자료/응급플랫폼논문/중증질환병원.xlsx',
             index=False)

In [78]:
real_em_all = pd.read_csv('/Users/subikim/Desktop/연구실/연구실자료/응급플랫폼논문/응급실실시간(전국)_위도경도결합.csv',
                         index_col=[0])

In [72]:
real_em_jb = hspt.copy()

In [80]:
real_em_all.columns=['기관명','기관주소','위도','경도','응급실','응급실연락처','기관코드_x','응급실직통연락처',
                    'VENTI(소아)','인큐베이터(보육기)','소아당직연락처','내과중환자실','외과중환자실','정형외과입원실','신경과입원실',
                    '신경외과중환자실','약물중환자','화상중환자','외상중환자',
                    '구급차가용여부','조영촬영기가용','신경중환자',
                    '흉부중환자','CT가용','입원실','일반중환자','입력일시','MRI가용','신생중환자','수술실','인공호흡기가용',
                    '기관코드_x','일련번호']

In [89]:
real_em_all.head(10).to_excel('/Users/subikim/Desktop/연구실/연구실자료/응급플랫폼논문/응급실병원(전국).xlsx',
             index=False)

In [71]:
real_em_all.head()

,dutyName,dutyAddr,wgs84Lat,wgs84Lon,hvec,dutyTel3_x,hpid_x,hv1,hv10,hv11,hv12,hv2,hv3,hv4,hv5,hv6,hv7,hv8,hv9,hvamyn,hvangioayn,hvcc,hvccc,hvctayn,hvgc,hvicc,hvidate,hvmriayn,hvncc,hvoc,hvventiayn,phpid_x,rnum_x
0,의료법인강릉동인병원,강원도 강릉시 강릉대로419번길 42 (포남동),37.774326,128.907142,15.0,033-650-6105,A2200005,0.0,0,0,0.0,3.0,0.0,0.0,Y,0.0,Y,0.0,0.0,Y,N,0.0,0.0,Y,76.0,1.0,2.022022e+13,Y,0.0,3.0,Y,A2200005,1.0
1,강원도강릉의료원,강원도 강릉시 경강로 2007 (남문동),37.749310,128.888796,4.0,NaN,A2200011,0.0,0,0,0.0,0.0,0.0,0.0,Y,0.0,0,0.0,0.0,Y,N,0.0,0.0,Y,102.0,0.0,2.022022e+13,Y,0.0,2.0,Y,A2200011,2.0
2,강릉아산병원,강원도 강릉시 사천면 방동길 38,37.818427,128.857714,-5.0,033-610-3333,A2200008,0.0,Y,Y,0.0,3.0,2.0,0.0,Y,0.0,Y,0.0,0.0,Y,N,0.0,0.0,Y,79.0,0.0,2.022022e+13,Y,3.0,6.0,Y,A2200008,3.0
3,근로복지공단동해병원,강원도 동해시 하평로 11 (평릉동),37.532323,129.105856,7.0,033-530-3119,A2200038,0.0,0,0,0.0,0.0,0.0,0.0,Y,0.0,0,0.0,0.0,Y,N,0.0,0.0,Y,62.0,1.0,2.022022e+13,Y,0.0,2.0,Y,A2200038,4.0
4,의료법인 동해동인병원,강원도 동해시 하평로 26 (평릉동),37.530007,129.107404,0.0,033-530-0129,A2200003,0.0,0,0,0.0,0.0,0.0,0.0,Y,0.0,0,0.0,0.0,Y,N,0.0,0.0,Y,105.0,8.0,2.022022e+13,Y,0.0,0.0,Y,A2200003,5.0
